In [ ]:
import pandas as pd
import plotly.express as px

paths = {
    "circuits": "../data/circuits.csv",
    "constructor_results": "../data/constructor_results.csv",
    "constructor_standings": "../data/constructor_standings.csv",
    "constructors": "../data/constructors.csv",
    "driver_standings": "../data/driver_standings.csv",
    "drivers": "../data/drivers.csv",
    "lap_times": "../data/lap_times.csv",
    "pit_stops": "../data/pit_stops.csv",
    "qualifying": "../data/qualifying.csv",
    "races": "../data/races.csv",
    "results": "../data/results.csv",
    "seasons": "../data/seasons.csv",
    "sprint_results": "../data/sprint_results.csv",
    "status": "../data/status.csv"
}

# Load data and generate schema summaries
summaries = {}
for name, path in paths.items():
    df = pd.read_csv(path, nrows=5)
    summaries[name] = {
        "columns": df.columns.tolist(),
        "dtypes": df.dtypes.astype(str).to_dict(),
        "sample_rows": df.to_dict(orient="records")
    }

summaries.keys()


dict_keys(['circuits', 'constructor_results', 'constructor_standings', 'constructors', 'driver_standings', 'drivers', 'lap_times', 'pit_stops', 'qualifying', 'races', 'results', 'seasons', 'sprint_results', 'status'])

In [3]:
# Load the full relevant datasets
drivers_df = pd.read_csv(paths["drivers"])
constructors_df = pd.read_csv(paths["constructors"])
races_df = pd.read_csv(paths["races"])
results_df = pd.read_csv(paths["results"])

# Merge drivers with results
driver_results = results_df.merge(drivers_df, on="driverId", how="left")

# Merge constructors
driver_results = driver_results.merge(constructors_df, on="constructorId", how="left")

# Merge races to get race metadata
driver_results = driver_results.merge(races_df, on="raceId", how="left")

# Select and rename for clarity
driver_results_cleaned = driver_results[[
    "raceId", "year", "round", "name_x", "date", "driverId", "forename", "surname",
    "constructorId", "name_y", "grid", "position", "points"
]].rename(columns={
    "name_x": "race_name",
    "name_y": "constructor_name"
})


driver_results_cleaned.head()

,raceId,year,round,race_name,date,driverId,forename,surname,constructorId,constructor_name,grid,position,points
0,18,2008,1,McLaren,2008-03-16,1,Lewis,Hamilton,1,Australian Grand Prix,1,1,10.0
1,18,2008,1,BMW Sauber,2008-03-16,2,Nick,Heidfeld,2,Australian Grand Prix,5,2,8.0
2,18,2008,1,Williams,2008-03-16,3,Nico,Rosberg,3,Australian Grand Prix,7,3,6.0
3,18,2008,1,Renault,2008-03-16,4,Fernando,Alonso,4,Australian Grand Prix,11,4,5.0
4,18,2008,1,McLaren,2008-03-16,5,Heikki,Kovalainen,1,Australian Grand Prix,3,5,4.0


In [4]:
# Clean the dataset for analysis
df = driver_results_cleaned.copy()

# Convert 'position' to numeric, coercing non-numeric values (like 'R', 'DQ') to NaN
df["position"] = pd.to_numeric(df["position"], errors="coerce")

# Convert 'date' to datetime
df["date"] = pd.to_datetime(df["date"])

# Top 10 drivers by total points
top_drivers = (
    df.groupby(["driverId", "forename", "surname"])["points"]
    .sum()
    .reset_index()
    .sort_values(by="points", ascending=False)
    .head(10)
)

top_drivers


,driverId,forename,surname,points
0,1,Lewis,Hamilton,4820.5
19,20,Sebastian,Vettel,3098.0
828,830,Max,Verstappen,2912.5
3,4,Fernando,Alonso,2329.0
7,8,Kimi,Räikkönen,1873.0
820,822,Valtteri,Bottas,1788.0
2,3,Nico,Rosberg,1594.5
813,815,Sergio,Pérez,1585.0
29,30,Michael,Schumacher,1566.0
842,844,Charles,Leclerc,1363.0


In [11]:
# Load your cleaned data here instead if not continuing from above
# df = pd.read_csv("your_cleaned_driver_data.csv")

# Assuming df is already cleaned as before
filtered_data = df.copy()
filtered_data["driver_name"] = filtered_data["forename"] + " " + filtered_data["surname"]
filtered_data = filtered_data.dropna(subset=["position", "grid"])
filtered_data["grid"] = pd.to_numeric(filtered_data["grid"], errors="coerce")
filtered_data["position"] = pd.to_numeric(filtered_data["position"], errors="coerce")

# Plot
fig = px.scatter(
    filtered_data.sample(n=1000, random_state=42),
    x="grid",
    y="position",
    color="driver_name",
    title="Grid Position vs Final Position for Top 10 Drivers",
    labels={"grid": "Starting Grid Position", "position": "Final Race Position"},
    hover_data=["race_name", "year"]
)
fig.update_yaxes(autorange="reversed")  # Better placements at the top
fig.show()


In [10]:
# Assuming results_df, constructors_df, races_df are already loaded
results_merged = results_df.merge(constructors_df, on="constructorId", how="left")
results_merged = results_merged.merge(races_df[["raceId", "year"]], on="raceId", how="left")

constructor_yearly = (
    results_merged.groupby(["year", "name"])["points"]
    .sum()
    .reset_index()
    .rename(columns={"name": "constructor_name"})
)

filtered_data = constructor_yearly.copy()

fig = px.line(
    filtered_data,
    x="year",
    y="points",
    color="constructor_name",
    title="Top Constructors Performance Over Time (Corrected)",
    markers=True
)
fig.show()


In [ ]:
# Group and prepare data
driver_yearly = (
    df.groupby(["year", "driverId", "forename", "surname"])["points"]
    .sum()
    .reset_index()
)
driver_yearly["driver_name"] = driver_yearly["forename"] + " " + driver_yearly["surname"]

filtered = driver_yearly.copy()

# Plot
fig = px.line(
    filtered,
    x="year",
    y="points",
    color="driver_name",
    title="Top Drivers Performance Over Time (Total Points per Season)",
    markers=True
)
fig.show()
